In [33]:
import requests
import shutil
from IPython.display import Image
from IPython.core.display import HTML 

# API code to access the genre and movie poster path of your favorite movie
One of my favorite movies is `The Departed`. Let's access the information for it via the TMDB API.

Following the instructions at https://developers.themoviedb.org/3/getting-started, this is pretty straightforward.

## Accessing the genre of The Departed

In [17]:
# API Key (Harry's)
params = {
            'api_key': '5eef6423b6f7cea2f5b6cae0838e38f9',
            'query': 'The Departed' 
         }

r = requests.get('https://api.themoviedb.org/3/search/movie?api_key=5eef6423b6f7cea2f5b6cae0838e38f9&query=The+Departed').json()
r

{u'page': 1,
 u'results': [{u'adult': False,
   u'backdrop_path': u'/8Od5zV7Q7zNOX0y9tyNgpTmoiGA.jpg',
   u'genre_ids': [18, 53, 80],
   u'id': 1422,
   u'original_language': u'en',
   u'original_title': u'The Departed',
   u'overview': u"To take down South Boston's Irish Mafia, the police send in one of their own to infiltrate the underworld, not realizing the syndicate has done likewise. While an undercover cop curries favor with the mob kingpin, a career criminal rises through the police ranks. But both sides soon discover there's a mole among them.",
   u'popularity': 5.297378,
   u'poster_path': u'/tGLO9zw5ZtCeyyEWgbYGgsFxC6i.jpg',
   u'release_date': u'2006-10-05',
   u'title': u'The Departed',
   u'video': False,
   u'vote_average': 7.8,
   u'vote_count': 3271},
  {u'adult': False,
   u'backdrop_path': None,
   u'genre_ids': [27, 18],
   u'id': 321724,
   u'original_language': u'ko',
   u'original_title': u'\u5996\u82b1\uc758 \u50b3\u8aaa',
   u'overview': u"A devoted son hears 

The first result is the one we want, so let's extract the movie's id in order to look up the genre of the film.

In [20]:
the_departed_id = r['results'][0]['id']
# Sanity check to see if we got the right one
the_departed_id

1422

In [29]:
# Now let's access the film's information by id in order to lookup the genre
r = requests.get('https://api.themoviedb.org/3/movie/' + str(the_departed_id) + '?api_key=5eef6423b6f7cea2f5b6cae0838e38f9').json()
genres_list = r['genres']

# Unpack the names of the genres
for genre in genres_list:
    print genre['name']

Drama
Thriller
Crime


## Accessing the poster for The Departed

In [34]:
poster_path = r['poster_path']
url = 'http://image.tmdb.org/t/p/w500/' + poster_path
Image(url=url)

# Genre for this movie listed by TMDb and IMDb

Let's compare the genres fetched from TMDb with those from IMDb.
http://www.imdb.com/title/tt0407887/

IMDb lists the genres:
Crime, Drama, Thriller

We have a match!

# A list of the 10 most popular movies of 2016 from TMDb and their genre obtained via the API

In [44]:
popular_r = requests.get('https://api.themoviedb.org/3/discover/movie?' + 'api_key=5eef6423b6f7cea2f5b6cae0838e38f9' + '&primary_release_year=2016').json()

# Print names of most popular movies
print "Names of most popular films in 2016:"
titles = []
# Get list of ids for genre lookup
ids = []
for i, film in enumerate(popular_r['results'][:10]):
    titles.append(film['original_title'])
    ids.append(film['id'])
    print i+1, film['original_title']

print "\n"
    
# Look up genres for each of the films
for i, id in enumerate(ids):
    print "Genres for", titles[i], ':' 
    
    # Now let's access the film's information by id in order to lookup the genre
    r = requests.get('https://api.themoviedb.org/3/movie/' + str(id) + '?api_key=5eef6423b6f7cea2f5b6cae0838e38f9').json()
    genres_list = r['genres']

    # Unpack the names of the genres
    for genre in genres_list:
        print genre['name']
    
    print "\n"

Names of most popular films in 2016:
1 Sing
2 Fantastic Beasts and Where to Find Them
3 Finding Dory
4 Rogue One: A Star Wars Story
5 Arrival
6 Deadpool
7 Doctor Strange
8 Underworld: Blood Wars
9 Captain America: Civil War
10 Zootopia


Genres for Sing :
Animation
Comedy
Drama
Family
Music


Genres for Fantastic Beasts and Where to Find Them :
Adventure
Action
Fantasy


Genres for Finding Dory :
Adventure
Animation
Comedy
Family


Genres for Rogue One: A Star Wars Story :
Action
Drama
Science Fiction
War


Genres for Arrival :
Drama
Science Fiction


Genres for Deadpool :
Action
Adventure
Comedy
Romance


Genres for Doctor Strange :
Action
Adventure
Fantasy
Science Fiction


Genres for Underworld: Blood Wars :
Action
Horror


Genres for Captain America: Civil War :
Action
Science Fiction


Genres for Zootopia :
Animation
Adventure
Family
Comedy


